In [2]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

In [5]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [6]:
img_t

tensor([[[-0.9448,  0.8272, -0.0574, -0.7360, -0.4515],
         [-0.7626, -1.0390,  1.9541,  1.0841,  1.0303],
         [ 0.1229, -0.7595,  0.0538,  0.2751, -0.1366],
         [-0.1119, -0.8353, -0.4251, -0.7999,  1.3960],
         [ 0.4343,  0.9271, -0.2029, -0.6574, -0.7690]],

        [[-0.0320,  0.9168, -0.3422, -1.0679, -0.2962],
         [-0.1672, -1.1881,  0.0806,  0.9384, -0.5554],
         [-0.1377, -0.1681, -0.2823,  0.5124,  0.6639],
         [ 0.5978,  1.0214, -1.5781, -0.0088,  0.0455],
         [ 0.9991, -1.8666, -0.6931, -2.1692, -0.7349]],

        [[ 0.3852,  1.1697,  0.5582, -1.8670, -0.3267],
         [ 0.6997,  1.9996, -0.2048,  0.5585, -0.9976],
         [ 0.2968,  0.6687, -1.3387,  0.4812,  0.7710],
         [-1.2946, -0.6068, -1.6145,  0.7513, -0.4532],
         [ 0.7982,  0.9499, -0.8110,  1.6243, -0.7544]]])

In [7]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [ ]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [8]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [9]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [14]:
weights_named.shape

torch.Size([3])

In [17]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [20]:
weights_aligned.names

('channels', 'rows', 'columns')

In [18]:
img_named.shape, weights_aligned.shape

(torch.Size([3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [ ]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [ ]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))